In [2]:
{"role": "assistant", "content": "안녕하세요 공간대여챗봇입니다. 무엇을 도와드릴까요?"}

{'role': 'assistant', 'content': '안녕하세요 공간대여챗봇입니다. 무엇을 도와드릴까요?'}

In [1]:
pip install --upgrade google-api-python-client

  Using cached httplib2-0.22.0-py3-none-any.whl.metadata (2.6 kB)
   ---------------------------------------- 0.0/12.6 MB ? eta -:--:--
   - -------------------------------------- 0.5/12.6 MB 11.1 MB/s eta 0:00:02
   -- ------------------------------------- 0.7/12.6 MB 9.2 MB/s eta 0:00:02
   --- ------------------------------------ 1.1/12.6 MB 7.8 MB/s eta 0:00:02
   --- ------------------------------------ 1.2/12.6 MB 7.0 MB/s eta 0:00:02
   ----- ---------------------------------- 1.6/12.6 MB 7.2 MB/s eta 0:00:02
   ----- ---------------------------------- 1.7/12.6 MB 6.8 MB/s eta 0:00:02
   ------ --------------------------------- 2.1/12.6 MB 6.8 MB/s eta 0:00:02
   ------- -------------------------------- 2.4/12.6 MB 6.5 MB/s eta 0:00:02
   -------- ------------------------------- 2.7/12.6 MB 6.6 MB/s eta 0:00:02
   --------- ------------------------------ 3.0/12.6 MB 6.3 MB/s eta 0:00:02
   ---------- ----------------------------- 3.2/12.6 MB 6.4 MB/s eta 0:00:02
   ----------- -


[notice] A new release of pip is available: 24.1.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [9]:
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
import pandas as pd

In [ ]:
def read_google_sheet(sheet_id: str, service_account_path: str, sheet_name: str = None):
    """
    Google Sheets 데이터를 읽어오는 단일 함수.
    
    :param sheet_id: Google Spreadsheet ID
    :param service_account_path: 서비스 계정 키 파일의 경로 (JSON)
    :param sheet_name: 읽고자 하는 시트 이름 (None이면 모든 시트 이름 출력)
    :return: 특정 시트 데이터 또는 모든 시트 이름 리스트
    """
    # Google API 서비스 계정 인증
    credentials = Credentials.from_service_account_file(
        service_account_path,
        scopes=["https://www.googleapis.com/auth/spreadsheets.readonly"]
    )
    service = build('sheets', 'v4', credentials=credentials)
    
    # 스프레드시트 메타데이터 가져오기
    spreadsheet = service.spreadsheets().get(spreadsheetId=sheet_id).execute()
    sheets = spreadsheet.get('sheets', [])
    
    # 시트 이름 가져오기
    sheet_names = [sheet['properties']['title'] for sheet in sheets]
    
    if not sheet_name:  # 시트 이름이 없으면 전체 시트 이름 반환
        return sheet_names
    
    # 특정 시트 데이터 가져오기
    if sheet_name not in sheet_names:
        raise ValueError(f"Sheet '{sheet_name}' does not exist in the spreadsheet.")
    
    range_name = f"{sheet_name}"  # 전체 시트를 읽기 위해 시트 이름만 사용
    result = service.spreadsheets().values().get(
        spreadsheetId=sheet_id,
        range=range_name
    ).execute()
    
    # 예제: Google Sheets API로부터 가져온 데이터 변환 후
    df = pd.DataFrame(result.get('values', []))  # 데이터프레임 생성
    df.columns = df.loc[0]  # 첫 번째 행을 열 이름으로 설정
    df = df[1:]  # 첫 번째 행 제거
    df = df.set_index('구 분')  # '구분' 열을 인덱스로 설정

        # 1. 빈 셀 확인 (비어 있는 값: None, NaN, 빈 문자열 등)
    empty_cells = df.isna() | (df == "")  # None이나 빈 문자열을 확인

    # 2. 빈 셀의 인덱스와 컬럼 이름 추출
    empty_locations = empty_cells.stack()[empty_cells.stack()].index.tolist()

    # 반복문을 통해 딕셔너리에 값 추가
    empty_space_dict = {}

    for items in empty_locations:
        key = items[0]
        value = items[1]
        
        # 키가 이미 딕셔너리에 존재하면 리스트에 추가, 아니면 새 리스트 생성
        if key in empty_space_dict:
            empty_space_dict[key].append(value)
        else:
            empty_space_dict[key] = [value]

    return empty_space_dict

In [ ]:
import os

sheet_id = os.getenv('GOOGLE_SHEET_ID')
service_account_path = os.getenv('GOOGLE_SERVICE_ACCOUNT_PATH')

sheet_name = '11월04일(월)'

empty_space_dict = read_google_sheet(sheet_id, service_account_path, sheet_name)

In [ ]:
from flask import Flask, request, jsonify
from googleapiclient.discovery import build
from google.oauth2.service_account import Credentials
import pandas as pd

app = Flask(__name__)

In [ ]:
def read_google_sheet(sheet_id, service_account_path, sheet_name):
    """
    Google Sheets 데이터를 읽고 빈 셀 정보를 반환하는 함수
    """
    # 서비스 계정 인증
    credentials = Credentials.from_service_account_file(
        service_account_path,
        scopes=["https://www.googleapis.com/auth/spreadsheets.readonly"]
    )
    service = build('sheets', 'v4', credentials=credentials)
    
    # 스프레드시트 메타데이터 가져오기
    spreadsheet = service.spreadsheets().get(spreadsheetId=sheet_id).execute()
    sheets = spreadsheet.get('sheets', [])
    sheet_names = [sheet['properties']['title'] for sheet in sheets]

    if sheet_name not in sheet_names:
        raise ValueError(f"Sheet '{sheet_name}' does not exist in the spreadsheet.")
    
    # 특정 시트 데이터 가져오기
    range_name = f"{sheet_name}"
    result = service.spreadsheets().values().get(
        spreadsheetId=sheet_id,
        range=range_name
    ).execute()
    
    # 데이터프레임 변환
    df = pd.DataFrame(result.get('values', []))
    df.columns = df.loc[0]  # 첫 번째 행을 열 이름으로 설정
    df = df[1:]  # 첫 번째 행 제거
    df = df.set_index(df.columns[0])  # 첫 번째 열을 인덱스로 설정
    
    # 빈 셀 위치 확인
    empty_cells = df.isna() | (df == "")
    empty_locations = empty_cells.stack()[empty_cells.stack()].index.tolist()

    # 빈 셀 딕셔너리 생성
    empty_space_dict = {}
    for items in empty_locations:
        key = items[0]
        value = items[1]
        if key in empty_space_dict:
            empty_space_dict[key].append(value)
        else:
            empty_space_dict[key] = [value]

    return empty_space_dict

@app.route('/read-google-sheet/<sheet_name>', methods=['GET'])
def handle_read_google_sheet(sheet_name):
    """
    Google Sheets 데이터를 읽고 빈 셀 정보를 반환하는 API
    """
    try:
        # Google Sheets 데이터 읽기
        result = read_google_sheet(DEFAULT_SHEET_ID, SERVICE_ACCOUNT_PATH, sheet_name)
        return jsonify(result), 200
    except ValueError as ve:
        return jsonify({"error": str(ve)}), 400
    except Exception as e:
        return jsonify({"error": "An unexpected error occurred", "details": str(e)}), 500

if __name__ == '__main__':
    app.run(host='0.0.0.0', port=5000, debug=True)
